In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

**Load Dataset**

In [37]:
ds = pd.read_csv("Churn_Modelling.csv")
ds.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Preprocessing**

In [38]:
ds = ds.iloc[:, 3:]

In [39]:
ds.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [40]:
encoder_gender = LabelEncoder()
ds["Gender"] = encoder_gender.fit_transform(ds["Gender"])

In [41]:
ds["Gender"].unique()

array([0, 1])

In [42]:
ds["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [43]:
## One hot encoding for Geography
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()
geography = one_hot_encoder.fit_transform(ds["Geography"].values.reshape(-1, 1)).toarray()

In [44]:
ds.drop("Geography", axis=1, inplace=True)

geography = pd.DataFrame(geography, columns=["France", "Germany", "Spain"])
ds = pd.concat([ds, geography], axis=1)

In [45]:
ds.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [46]:
## Saving the encoder and scaler
with open("gender_encoder.pkl", "wb") as file:
    pickle.dump(encoder_gender, file)

with open("one_hot_encoder_geo.pkl", "wb") as file:
    pickle.dump(one_hot_encoder, file)

In [47]:
ds.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Germany,Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [48]:
X = ds.drop("Exited", axis=1)
Y = pd.DataFrame(ds["Exited"])

In [49]:
Y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


**Split the data**

In [50]:
X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

**Scaling**

In [51]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [52]:
with open("scalar.pkl", "wb") as file:
    pickle.dump(scalar, file)

In [53]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [54]:
X_train.shape[1]

12

In [55]:
## Building the ANN model
model = Sequential([
    ##input shape
    Input(shape=(X_train.shape[1],)),
    ##hidden layer1
    Dense(64, activation="relu"),
    ##hidden layer2
    Dense(32, activation="relu"),
    ##output layer
    Dense(1, activation="sigmoid")
])

In [56]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)


In [58]:
##Compiling the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["accuracy"])

In [59]:
## Tensorboard
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [60]:
## Early stopping
early_stopping_callback = EarlyStopping(monitor="val_loss" ,patience=5, restore_best_weights=True)

In [61]:
## Training the model
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, callbacks=[early_stopping_callback, tensorflow_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8030 - loss: 0.4427 - val_accuracy: 0.8600 - val_loss: 0.3525
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8523 - loss: 0.3577 - val_accuracy: 0.8595 - val_loss: 0.3418
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8584 - loss: 0.3445 - val_accuracy: 0.8585 - val_loss: 0.3649
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8546 - loss: 0.3462 - val_accuracy: 0.8595 - val_loss: 0.3542
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8631 - loss: 0.3421 - val_accuracy: 0.8600 - val_loss: 0.3389
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8758 - loss: 0.3153 - val_accuracy: 0.8565 - val_loss: 0.3447
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8637 - loss: 0.3366 - val_accuracy: 0.8545 - val_loss: 0.3518
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8628 - loss: 0.3434 - val_accu

In [62]:
model.save("model.h5")

In [63]:
## Load Tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [66]:
%tensorboard --logdir logs/fit20241226-155118

Reusing TensorBoard on port 6007 (pid 15680), started 0:00:23 ago. (Use '!kill 15680' to kill it.)

In [68]:
one_hot_encoder.categories_[0]

array(['France', 'Germany', 'Spain'], dtype=object)

In [69]:
encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [72]:
ds["Age"].max()

92

In [73]:
ds["NumOfProducts"].max()

4